# 3b. Enrich Stream
  --------------------------------------------------------------------

Receive a stream of events from from `incoming-events-stream` and enrich the relevant events with socioeconomic data by looking up the enrichment table. The enriched data is streamed out to `enriched-events-stream`.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

You can change the enrichments and add additional enrichments by changing the `enrich_event` method.

## Create and Test a Local Function 
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
%run config.py

In [ ]:
input_stream_url = path.join(WEB_API_USERS, STREAM_CONFIGS['incoming-events-stream']['path']) + "@enrichstream"

#### Functions imports

In [ ]:
# nuclio: start-code

In [ ]:
import os
import json
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [ ]:
%nuclio cmd -c pip install v3io

In [ ]:
%%nuclio config
spec.build.baseImage = "mlrun/ml-models"


## Function code

In [ ]:
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    CONTAINER = os.getenv('CONTAINER')
    OUTPUT_STREAM_PATH = os.getenv('MDWS_ES_OUTPUT_STREAM_PATH')
    PARTITION_ATTR = os.getenv('MDWS_PARTITION_ATTR')
    ENRICHMENT_TABLE_PATH = os.getenv('MDWS_ENRICHMENT_TABLE_PATH')
    ENRICHMENT_KEY = os.getenv('ENRICHMENT_KEY')
    v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)
    
    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'partition_attr', PARTITION_ATTR)
    setattr(context, 'container', CONTAINER)
    setattr(context, 'output_stream_path', OUTPUT_STREAM_PATH)
    
    setattr(context, 'enrichment_table_path', ENRICHMENT_TABLE_PATH)
    setattr(context, 'enrichment_key', ENRICHMENT_KEY)


def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
        
    partition_key = event_dict.get(context.partition_attr)
    
    record = {}
    if event_dict['event_type'] == 'registration':
        enriched_event = enrich_event(context, event_dict)
        record = event_to_record(enriched_event, partition_key)
    else:
        record = event_to_record(event_dict, partition_key)
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp.status_code


def enrich_event(context, event_dict):
    if context.enrichment_key in event_dict:
        enrichment_key_value = event_dict[context.enrichment_key]
        resp = context.v3io_client.get_item(container=context.container, 
                                            path=os.path.join(context.enrichment_table_path, str(enrichment_key_value)),
                                           raise_for_status=v3io.dataplane.RaiseForStatus.never)
        if 200 <= resp.status_code <= 299:
            enriched_event = dict(event_dict, **resp.output.item)
            context.logger.info_with('Event was enriched', enriched_event=enriched_event)
            return enriched_event
        else:
            context.logger.debug_with("Couldn't enrich event", 
                                      enrichment_key_value=enrichment_key_value,
                                      response_status=resp.status_code, 
                                      response_body=resp.body.decode('utf-8'))
            return event_dict
    else:
        return event_dict

    
def event_to_record(event_dict, partition_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [ ]:
# nuclio: end-code
# marks the end of a code section

## Test locally

In [ ]:
event = nuclio.Event(body=b'{"user_id" : 111111 , "event_type": "registration", "postcode": 11014}')
init_context(context)
handler(context, event)

# MLRUN

In [ ]:
%store -r project_name
%store -r project_path
%store -r artifact_path
%store -r project

In [ ]:
from mlrun import code_to_function


gen_func = code_to_function(name='enrich', kind = 'nuclio')
project.set_function(gen_func)
enrich = project.func('enrich')
enrich.set_envs({'V3IO_ACCESS_KEY': V3IO_ACCESS_KEY, 'CONTAINER': "users", 'MDWS_ES_OUTPUT_STREAM_PATH': STREAM_CONFIGS['enriched-events-stream']['path'], 'MDWS_PARTITION_ATTR':PARTITION_ATTR,'MDWS_ENRICHMENT_TABLE_PATH':ENRICHMENT_TABLE_PATH,'ENRICHMENT_KEY':"postcode"})
enrich.add_trigger('incoming', nuclio.triggers.V3IOStreamTrigger(url=input_stream_url, access_key=V3IO_ACCESS_KEY,maxWorkers=10))


In [ ]:
enrich.deploy()